In [7]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
# from linearmodels.panel.data import PanelData
# from linearmodels.panel import PanelOLS, PooledOLS, RandomEffects, compare
import matplotlib.pyplot as plt
import pyblp

In [8]:
save=0
product_data=pd.read_csv("../../data/merged/len3_ndb_blp_imputed_lags.csv",encoding="shift-jis",index_col=0)
product_data["market_ids"]=product_data["薬効分類"].astype(int).astype(str)+"-"+product_data["year"].astype(str)
product_data["generic_share_q"].fillna(0,inplace=True)
product_data["generic_share_r"].fillna(0,inplace=True)
product_data["lag_sum_quantity"]=np.log(product_data["lag_sum_quantity"]+0.001)
product_data.rename(columns={"薬価":"prices","r_share":"shares",
                             "後発品区分":"generic",
                    # "elasped_0":"demand_instruments0",
                    # "elasped_1":"demand_instruments1",
                    # 'elasped_2': 'demand_instruments2',
                    # 'elasped_3': 'demand_instruments3',
                    # 'elasped_4': 'demand_instruments4',
                    # 'elasped_5': 'demand_instruments5',
                    # 'elasped_6': 'demand_instruments6',
                    # 'elasped_7': 'demand_instruments7',
                    # 'elasped_8': 'demand_instruments8',
                    # 'elasped_9': 'demand_instruments9',
                    # "year":"demand_instruments2",
                    # "generic_per":"demand_instruments4",
                    # "mean_price":"demand_instruments0",
                    # "otc_dom":"demand_instruments2",
                    "lag_mean_price":"demand_instruments0",
                    # "generic_share_q":"demand_instruments7",
                    "lag_sum_quantity":"demand_instruments1",
                    "generic_share_r":"demand_instruments2"
                    },inplace=True)
product_data=product_data.loc[product_data["shares"]>0]
# data["shares"]=data["r_share"]
product_data.head()

,薬効分類,薬効分類名称,医薬品コード,医薬品名,薬価基準収載医薬品コード,prices,generic,総計,year,in_hospital,...,id_449.0,id_625.0,id_629.0,id_799.0,q_share,shares,generic_share_q,demand_instruments2,y_sq,market_ids
0,114.0,解熱鎮痛消炎剤,620007096.0,ボルタレン錠２５ｍｇ,1147002F1560,13.1,0.0,2.489539e+07,2014.0,1,...,0.0,0.0,0.0,0.0,0.007258,0.002068,0.236142,0.053889,4056196.0,114-2014.0
1,114.0,解熱鎮痛消炎剤,620007095.0,ボルタレンＳＲカプセル３７．５ｍｇ,1147002N1174,23.2,0.0,1.024590e+07,2014.0,1,...,0.0,0.0,0.0,0.0,0.002987,0.001507,0.236142,0.053889,4056196.0,114-2014.0
2,114.0,解熱鎮痛消炎剤,620007096.0,ボルタレン錠２５ｍｇ,1147002F1560,13.1,0.0,5.025912e+07,2014.0,0,...,0.0,0.0,0.0,0.0,0.014653,0.004175,0.236142,0.053889,4056196.0,114-2014.0
3,114.0,解熱鎮痛消炎剤,620007095.0,ボルタレンＳＲカプセル３７．５ｍｇ,1147002N1174,23.2,0.0,2.742935e+07,2014.0,0,...,0.0,0.0,0.0,0.0,0.007997,0.004036,0.236142,0.053889,4056196.0,114-2014.0
4,114.0,解熱鎮痛消炎剤,661140081.0,ボルタレンサポ５０ｍｇ,1147700J3084,63.1,0.0,5.339043e+06,2014.0,1,...,0.0,0.0,0.0,0.0,0.001557,0.002136,0.236142,0.053889,4056196.0,114-2014.0


## random coef

In [9]:
# product_data=product_data.loc[product_data["year"]==2015]
N=product_data.shape[0]
X1_formulation = pyblp.Formulation('prices+oral+in_hospital+generic',absorb='C(薬効分類)+C(year)',absorb_method="lsmr")
X2_formulation = pyblp.Formulation('-1+I(-prices)+generic')
product_formulations = (X1_formulation, X2_formulation)
agent_formulation = pyblp.Formulation('1')



# optim = pyblp.Optimization('nelder-mead',compute_gradient=False)
# optim = pyblp.Optimization('slsqp')


In [10]:
N

25798

In [11]:
agent_num=200
agent_data=pd.DataFrame(np.ones((agent_num*N,1)))
ids=product_data["market_ids"].repeat(agent_num)
ids.reset_index(drop=True,inplace=True)
agent_data["market_ids"]=ids

In [12]:
mc_integration = pyblp.Integration('monte_carlo', size=agent_num, specification_options={'seed': 0})
mc_integration
# mc_problem = pyblp.Problem(product_formulations, product_data ,agent_formulation,agent_data,integration=mc_integration,rc_types=["log","linear"])
mc_problem = pyblp.Problem(product_formulations, product_data ,integration=mc_integration,rc_types=["log","linear"])
# mc_problem

Initializing the problem ...
Absorbing demand-side fixed effects ...
Initialized the problem after 00:00:00.

Dimensions:
 T     N      I     K1    K2    MD    ED 
---  -----  -----  ----  ----  ----  ----
200  25798  40000   4     2     6     2  

Formulations:
       Column Indices:            0        1          2          3   
-----------------------------  -------  -------  -----------  -------
 X1: Linear Characteristics    prices    oral    in_hospital  generic
X2: Nonlinear Characteristics  -prices  generic                      


In [15]:
optim = pyblp.Optimization('l-bfgs-b',{'gtol': 1e-4})
# optim = pyblp.Optimization('nelder-mead',compute_gradient=False)
# optim = pyblp.Optimization('slsqp')
lb=-10
ub=10
# results1 = mc_problem.solve(sigma=np.eye(2),pi=[[1],[0]],sigma_bounds = [((lb, lb), (lb, lb)),
#                ((ub, ub), (ub, ub))],pi_bounds=((0,0),(10,10)),optimization=optim)
# in case of agent data exists
# results1 = mc_problem.solve(sigma=np.eye(2),pi=[[1],[0]],sigma_bounds = (((0,0),(0,None)),((None,None),(None,None))) ,pi_bounds=((0,None),(None,None)),optimization=optim)
# without agent data
results1 = mc_problem.solve(sigma=np.eye(2),sigma_bounds = (((0,0),(0,None)),((None,None),(None,None))) ,optimization=optim)
# results1

Solving the problem ...

Nonlinear Coefficient Initial Values:
Types:  |  Sigma:      -prices        generic   
------  |  -------  -------------  -------------
 Log    |  -prices  +1.000000E+00               
Linear  |  generic  +0.000000E+00  +1.000000E+00

Nonlinear Coefficient Lower Bounds:
Types:  |  Sigma:      -prices        generic   
------  |  -------  -------------  -------------
 Log    |  -prices  +0.000000E+00               
Linear  |  generic  +0.000000E+00      -INF     

Nonlinear Coefficient Upper Bounds:
Types:  |  Sigma:      -prices        generic   
------  |  -------  -------------  -------------
 Log    |  -prices      +INF                    
Linear  |  generic  +0.000000E+00      +INF     

Starting optimization ...


At least one error was encountered. As long as the optimization routine does not get stuck at values of theta that give rise to errors, this is not necessarily a problem. If the errors persist or seem to be impacting the optimization results, con

In [16]:
import datetime

# Get the current date and time
current_time = datetime.datetime.now()

# Format the time as a string
time_string = current_time.strftime("%Y-%m-%d_%H-%M-%S")
# save=1
if save:
    results1.to_pickle(f"./results/RCL_len3_{time_string}.pkl")

In [ ]:
# import sys
# filename = "./results/blp_0904_2015_slsqp.txt"
# with open(filename, "w") as file:
#     # Redirect the print output to the file
#     original_stdout = sys.stdout
#     sys.stdout = file
    
#     # Print the contents of the ProblemResults object
#     print(results1)
    
#     # Reset the print output back to the original stdout
#     sys.stdout = original_stdout

In [ ]:
# pr_integration = pyblp.Integration('product', size=5)
# # pr_problem = pyblp.Problem(product_formulations, product_data,agent_formulation,agent_data,integration=pr_integration,rc_types=["log","linear"])
# pr_problem = pyblp.Problem(product_formulations, product_data,agent_formulation,agent_data,integration=pr_integration,rc_types=["log","linear"])
# optim = pyblp.Optimization('bfgs',{'gtol': 1e-4})
# # with pyblp.parallel(3):
# results2 = pr_problem.solve(sigma=np.eye(2),pi=[[1],[1]], optimization=optim)
# results2
